# YOLOv10 Garbage Detection – Baseline Training (v1)

This notebook performs baseline training of a YOLOv10 object detection model
using a pretrained YOLOv10s checkpoint. The resulting model checkpoint is later
used as the starting point for fine-tuning experiments (v2).

## Environment Setup

This section prepares the YOLOv10 training environment and installs
all required dependencies. The experiment is conducted using Google Colab.

In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10
!pip install -r requirements.txt

Cloning into 'yolov10'...
remote: Enumerating objects: 20338, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20338 (delta 2), reused 0 (delta 0), pack-reused 20334 (from 2)
Receiving objects: 100% (20338/20338), 11.10 MiB | 26.50 MiB/s, done.
Resolving deltas: 100% (14354/14354), done.
/content/yolov10
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.0.1


In [ ]:
!pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 109.0 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


## Environment Check

Verify PyTorch installation and ensure the environment is properly configured.

In [ ]:
import torch

print("PyTorch version:", torch.__version__)

2.9.0+cu126


## Dataset (Version 1)

This experiment uses the initial version of the garbage dataset.
The dataset is provided in YOLO format and serves as the baseline
for subsequent model improvement.

In [ ]:
!unzip Trash.v1i.yolov8.zip

Archive:  Trash.v1i.yolov8.zip
   creating: Trash.v1i.yolov8/
  inflating: Trash.v1i.yolov8/data.yaml  
  inflating: Trash.v1i.yolov8/README.dataset.txt  
  inflating: Trash.v1i.yolov8/README.roboflow.txt  
   creating: Trash.v1i.yolov8/train/
   creating: Trash.v1i.yolov8/train/images/
  inflating: Trash.v1i.yolov8/train/images/32911402747_496ac0b26e_o_jpg.rf.c6221073c81ed777ed70adcee2c421d0.jpg  
  inflating: Trash.v1i.yolov8/train/images/32911404057_b139c11643_o_jpg.rf.9ba6948814510085de0bcef97c1c949a.jpg  
  inflating: Trash.v1i.yolov8/train/images/32911404467_75881a027e_o_jpg.rf.c1a8d3c89ace6cb9ffa75962d8e07fa6.jpg  
  inflating: Trash.v1i.yolov8/train/images/32911404617_b7c9ea03d6_o_jpg.rf.4654991a170c3c1ec47e86d7ee66f623.jpg  
  inflating: Trash.v1i.yolov8/train/images/32911405727_69e1c9873d_o_jpg.rf.e5fad3e420262c2c00ec35cfd7ae981e.jpg  
  inflating: Trash.v1i.yolov8/train/images/32911411497_62842b1d69_o_jpg.rf.14fb2df76798983149253de94b62ba54.jpg  
  inflating: Trash.v1i.yolov

## Baseline Model Initialization

The baseline experiment starts from the official YOLOv10s pretrained checkpoint.
This model serves as the initial reference before fine-tuning on later versions
of the dataset.

In [ ]:
!wget https://github.com/THU-MIG/yolov10/releases/download/v1.0/yolov10s.pt

--2026-01-05 03:25:21--  https://github.com/THU-MIG/yolov10/releases/download/v1.0/yolov10s.pt
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/804788522/3024a54a-69ef-4ac0-9e8c-c3953b5c3ee1?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-05T04%3A10%3A32Z&rscd=attachment%3B+filename%3Dyolov10s.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-05T03%3A10%3A07Z&ske=2026-01-05T04%3A10%3A32Z&sks=b&skv=2018-11-09&sig=aMxGqu8B1fHEdx7MzPODgpeQw1rzPrafx2zCJNYBYZI%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NzU4NTMyMSwibmJmIjoxNzY3NTgzNTIxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmUud2lu

## Baseline Training

The YOLOv10s model is trained from scratch on the initial dataset (v1).
This baseline model establishes a performance reference and generates
a checkpoint used for fine-tuning in subsequent experiments.

In [ ]:
!yolo task=detect \
  mode=train \
  model=yolov10s.pt \
  data=/content/yolov10/Trash.v1i.yolov8/data.yaml \
  epochs=100 \
  imgsz=640 \
  batch=16

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov10/Trash.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

## Baseline Inference

The trained baseline model is evaluated on a sample image to
verify detection performance before further fine-tuning.

In [ ]:
!yolo task=detect \
  mode=predict \
  model=/content/yolov10/runs/detect/train2/weights/best.pt \
  source=/content/yolov10/46939957695_34122d087b_o.jpg \
  conf=0.25

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10s summary (fused): 106 layers, 7,219,935 parameters, 0 gradients, 21.4 GFLOPs

image 1/1 /content/yolov10/46939957695_34122d087b_o.jpg: 640x480 1 plastic, 43.1ms
Speed: 2.9ms preprocess, 43.1ms inference, 14.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/yolov10/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


## Save Training Artifacts

The trained model and experiment logs are archived for backup
and future fine-tuning experiments.

In [ ]:
!zip -r train2.zip runs/detect/train2

  adding: runs/detect/train2/ (stored 0%)
  adding: runs/detect/train2/labels.jpg (deflated 31%)
  adding: runs/detect/train2/val_batch2_labels.jpg (deflated 4%)
  adding: runs/detect/train2/val_batch0_pred.jpg (deflated 5%)
  adding: runs/detect/train2/train_batch0.jpg (deflated 1%)
  adding: runs/detect/train2/val_batch0_labels.jpg (deflated 5%)
  adding: runs/detect/train2/BoxP_curve.png (deflated 6%)
  adding: runs/detect/train2/train_batch2252.jpg (deflated 4%)
  adding: runs/detect/train2/args.yaml (deflated 53%)
  adding: runs/detect/train2/val_batch1_labels.jpg (deflated 4%)
  adding: runs/detect/train2/weights/ (stored 0%)
  adding: runs/detect/train2/weights/last.pt (deflated 9%)
  adding: runs/detect/train2/weights/best.pt (deflated 9%)
  adding: runs/detect/train2/results.csv (deflated 60%)
  adding: runs/detect/train2/BoxPR_curve.png (deflated 13%)
  adding: runs/detect/train2/train_batch2.jpg (deflated 1%)
  adding: runs/detect/train2/val_batch2_pred.jpg (deflated 4%)
  a

In [ ]:
!zip -r yolov10.zip /content/yolov10

  adding: content/yolov10/ (stored 0%)
  adding: content/yolov10/flops.py (deflated 46%)
  adding: content/yolov10/yolo11n.pt (deflated 11%)
  adding: content/yolov10/.pre-commit-config.yaml (deflated 60%)
  adding: content/yolov10/examples/ (stored 0%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/ (stored 0%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/main.cpp (deflated 59%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/inference.h (deflated 54%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/README.md (deflated 48%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/inference.cpp (deflated 70%)
  adding: content/yolov10/examples/YOLOv8-CPP-Inference/CMakeLists.txt (deflated 43%)
  adding: content/yolov10/examples/YOLOv8-OpenCV-ONNX-Python/ (stored 0%)
  adding: content/yolov10/examples/YOLOv8-OpenCV-ONNX-Python/main.py (deflated 62%)
  adding: content/yolov10/examples/YOLOv8-OpenCV-ONNX-Python/README.md (deflated 31%)
  adding: content/

## Conclusion

This notebook establishes a baseline YOLOv10 garbage detection model
trained on the initial dataset version. The resulting checkpoint
serves as the foundation for fine-tuning and performance improvements
in later experiments (v2).